In [ ]:
# !pip install datasets==2.16.1

In [1]:
import random
import re
from typing import List

from datasets import load_dataset, list_datasets, Dataset
from IPython.display import Image
from IPython.core.display import HTML
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Model, GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm

https://huggingface.co/docs/transformers/model_doc/gpt2

https://huggingface.co/docs/transformers/main_classes/tokenizer

In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")
print(f"Our device is {DEVICE}")

Our device is cuda:0


# Задание 1: интерпретируем attention

Вам даны attention-скоры для последовательности ниже. Необходимо определить позицию токена, на который больше всего "обращал внимание" токен на позиции 4 во второй attention head на последнем слое модели *(все вышеупомянутые числа- индексы!)* . Ответом должен быть индекс

In [20]:
tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')
model = GPT2LMHeadModel.from_pretrained('openai-community/gpt2', output_attentions=True)
model.eval()

The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
model_input = tokenizer("The quick brown fox jumps over the lazy dog.", return_tensors='pt')
model_input

{'input_ids': tensor([[  464,  2068,  7586, 21831, 18045,   625,   262, 16931,  3290,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

[Документация](https://huggingface.co/docs/transformers/v4.45.1/en/model_doc/gpt2#transformers.GPT2LMHeadModel.forward) к модели

In [ ]:
"""
YOUR IMPLEMENTAION HERE
"""

# Как работает генерация с т.з. кода под капотом:

In [7]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [8]:
TEXT_INPUT = "Парламент- это не место для"

In [9]:
inputs = tokenizer(TEXT_INPUT, return_tensors="pt")

In [10]:
for k, v in inputs.items():
  inputs[k] = v.to(DEVICE)

In [11]:
bare_model = GPT2Model.from_pretrained("gpt2")
bare_model.eval()
bare_model.to(DEVICE)
bare_outputs = bare_model(**inputs, output_hidden_states=True)

In [12]:
bare_model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [13]:
last_hidden_states = bare_outputs.last_hidden_state

Weight tying

In [4]:
Image(url= "https://lena-voita.github.io/resources/lectures/lang_models/practical/weight_tying_idea-min.png", width=1900, height=900)

In [14]:
logits = torch.matmul(
    last_hidden_states[-1][-1],
    bare_model.wte.weight.T
)

In [15]:
bare_probas = F.softmax(logits, dim=0)

In [16]:
torch.argmax(bare_probas)

tensor(12466, device='cuda:0')

# Упрощённая генерация:

In [17]:
llm_model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
llm_model.eval()
llm_model.to(DEVICE)
llm_outputs = llm_model(**inputs)

In [18]:
n_params = 0
for param in llm_model.parameters(recurse=True):
    n_params += param.numel()


n_params = str(n_params)
n_params = ",".join(
    [
        n_params[i: i+3]
        for i in range(
            len(n_params) - 3, -1, -3
        )
    ][::-1]
)
print(f"Number of parameters: {n_params}")

Number of parameters: 124,439,808


In [19]:
torch.argmax(llm_outputs.logits[-1][-1])

tensor(12466, device='cuda:0')

In [20]:
llm_probas = F.softmax(llm_outputs.logits[-1][-1], dim=0)

In [21]:
torch.allclose(bare_probas, llm_probas, rtol=1e-4)

False

# Задание 2: написать fine-tuning для языковой модели под набор данных:

Описание датасета можно найти тут: \
https://paperswithcode.com/dataset/rucos \
https://huggingface.co/datasets/RussianNLP/russian_super_glue

In [22]:
dataset = load_dataset("RussianNLP/russian_super_glue", name='rucos')

/home/unicornred/python_env/torch_gpu/lib64/python3.13/site-packages/datasets/load.py:1429: FutureWarning: The repository for RussianNLP/russian_super_glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/RussianNLP/russian_super_glue
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/72193 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7577 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7257 [00:00<?, ? examples/s]

In [23]:
RE_BAD_PATTERNS = re.compile("(@[a-z]+|\n)")

In [24]:
random_idx_from_train = random.randint(0, len(dataset['train']) - 1) # 66411

random_object = dataset['train'][random_idx_from_train]['passage']

filtered_random_object = RE_BAD_PATTERNS.sub(" ", random_object)
print(random_object)
print("*" * 20)
print(filtered_random_object)

Международная группа исследователей из России и США пришла к выводу, что города, находящиеся в зоне вечной мерзлоты в Восточной Сибири, могут разрушиться к 2050 году. Причиной является оттаивание грунтов из-за глобального потепления. Об этом сообщает издание The Siberian Times. Согласно оценке исследователей, через 35 лет несущая способность пород, в которых содержится вечная мерзлота, может уменьшиться на 75-95 процентов. Это приведет к деформации и обрушению как жилых, так и промышленных сооружений. Самые быстрые изменения коснутся городов Анадырь и Салехард — здания могут потерять устойчивость к середине 2020-х годов.
@highlight
Из вечной мерзлоты извлекли древний мозг щенка
@highlight
Гигантские воронки могут появиться на севере России
@highlight
Найдено объяснение сибирской аномалии
********************
Международная группа исследователей из России и США пришла к выводу, что города, находящиеся в зоне вечной мерзлоты в Восточной Сибири, могут разрушиться к 2050 году. Причиной явля

In [25]:
tokenizer.add_special_tokens({'pad_token': "<|endoftext|>"})

def texts_to_batch(texts: List[str]) -> torch.Tensor:
    clean_texts = [
        RE_BAD_PATTERNS.sub(" ", _["passage"]) for _ in texts
    ]
    tokenized_texts = tokenizer(
        text=clean_texts,
        return_tensors="pt",
        add_special_tokens=True,
        padding="max_length",
        truncation=True
      )
    return tokenized_texts

In [26]:
BATCH_SIZE = 2

train_dl = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=texts_to_batch
)

In [27]:
N_ITERATIONS = 1000

inputs = tokenizer("В прошлый четверг президенты Казахстана и России", return_tensors="pt")
for k, v in inputs.items():
    inputs[k] = v.to(DEVICE)

In [28]:
OUTPUT_SIZE = 40

In [29]:
llm_model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
llm_model.eval()
llm_model.to(DEVICE)
llm_outputs = llm_model(**inputs)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=llm_model.parameters(), lr=1e-6
)

cur_iteration = 0
for batch in train_dl:
    if cur_iteration == N_ITERATIONS:
        break

    llm_model.train()
    """
    YOUR IMPLEMENTAION HERE
    """
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()


    llm_model.eval()
    for n_beams in range(2, 5):
        beam_output = llm_model.generate(**inputs, max_new_tokens=OUTPUT_SIZE, num_beams=n_beams)
        print(f"Beam size={n_beams}")
        print(tokenizer.decode(beam_output[0], skip_special_tokens=True))
        print()

    print("*" * 20)
    cur_iteration += 1

# Задание 3: написать greedy search, сравнить результаты с имплементацией от transformers

In [ ]:
OUTPUT_SIZE = 40

In [ ]:
llm_model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
llm_model.eval()
llm_model.to(DEVICE)
llm_outputs = llm_model(**inputs)

In [ ]:
def convert_to_expected_input(input_ids, attention_mask):
    input_ids = torch.tensor(input_ids, device=DEVICE)
    attention_mask = torch.tensor(attention_mask, device=DEVICE)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }

In [ ]:
input_ids, attention_mask = inputs["input_ids"].tolist(), inputs["attention_mask"].tolist()
"""
input_ids должно содержать нагенерированные токены
"""

for _ in range(OUTPUT_SIZE):
    """
    YOUR IMPLEMENTAION HERE
    """

In [ ]:
llm_predictions = llm_model.generate(**inputs, max_new_tokens=OUTPUT_SIZE)

Ниже проверяем, что наивная имплементация совпадает с ожидаемой:

In [ ]:
assert input_ids[-1][-OUTPUT_SIZE:] == llm_predictions[-1][-OUTPUT_SIZE:].tolist()

In [ ]:
tokenizer.decode(llm_predictions[0], skip_special_tokens=True)